In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
pip install -q git+https://github.com/eaedk/testing-zindi-package.git

In [3]:
# Import Zindian class from the package
from zindi.user import Zindian
# create a user object
my_username = "Johannes"
user = Zindian(username = my_username)

In [4]:
user.select_a_challenge()                               # Select a Zindi challeng

In [5]:
user.download_dataset(destination="dataset") # Download the dataset of the selected challenge

In [7]:
!pip install -qU 'git+https://github.com/PyTorchLightning/lightning-flash.git#egg=lightning-flash[image]'

In [7]:
train = pd.read_csv('./dataset/Train.csv')
test = pd.read_csv('./dataset/Test.csv')
ss = pd.read_csv('./dataset/SampleSubmission.csv')

In [9]:
!unzip ./dataset/coral_image.zip

In [10]:
train.label.value_counts()

In [9]:
import glob

total_path = glob.glob('./coral_image/*.*')
# total_path

In [10]:
list_train = pd.DataFrame()
list_train['image_path'] = total_path
list_train['Image_ID'] = list_train['image_path'].apply(lambda x: x.split('/')[2])
list_train['Image_ID'] = list_train['Image_ID'].apply(lambda x: x.split(".")[0])
train = pd.merge(train, list_train, on=["Image_ID"]).drop('Image_ID', axis=1)
test = pd.merge(test, list_train, on=["Image_ID"])
train.head()

In [11]:
train = train[train['label']!='Soft Coral']
train = train[train['label']!='Other Invertebrates']
# train = train[train['label']!='Other']
train.label.value_counts()

In [12]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train['label'])
train['label'] = le.transform(train['label'])
# train['label'] = train['label'].astype(float)

In [13]:
train.label.value_counts()

In [19]:
train.head()

In [14]:
import torch
import flash
from flash.image import ImageClassificationData, ImageClassifier

In [61]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score

X = train[['image_path']]
y = train[['label']]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.5, 
                                                      random_state=3, 
                                                      shuffle=True)

In [16]:
X_train['label'] = y_train 

In [17]:
images = [i for i in X_train['image_path']]
label = [i for i in X_train['label']]




train_transform = {
    "post_tensor_transform": ApplyToKeys(
        "input", AlbumentationsAdapter(albumentations.HorizontalFlip(p=0.5))),}


train_transform = merge_transforms(default_transforms((128, 128)), train_transform)

datamodule = ImageClassificationData.from_files(
    train_files = images,
    train_targets = label,image_size=(128, 128),val_split=0.1,
#     train_transform=train_transform,
#     val_transform=transform,
#    test_transform=transform,

)


In [18]:
datamodule.show_train_batch()

In [22]:
import flash
from flash.image import ImageClassifier


# model = ImageClassifier(backbone="resnet18", num_classes=datamodule.num_classes,
#                         optimizer="Adam",
#                        lr_scheduler=("StepLR", {"step_size": 10}))
# backbone="resnext50_32x4d"
model = ImageClassifier(backbone="resnet18", num_classes=datamodule.num_classes,
                        optimizer="Adam")

In [23]:
trainer = flash.Trainer(max_epochs=3, gpus=torch.cuda.device_count())

In [24]:
%%time
trainer.finetune(model, datamodule=datamodule, strategy="freeze")

In [39]:
########## validation results confusion matrix
x = [i for i in X_valid['image_path']]


## the gpu crushes so i split the data in batches manually
import itertools

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

x1 = model.predict(list(batch(x, 2000))[0])
x2 = model.predict(list(batch(x, 2000))[1])
x3 = model.predict(list(batch(x, 2000))[2])
x4 = model.predict(list(batch(x, 2000))[3])
x5 = model.predict(list(batch(x, 2000))[4])
# x6 = model.predict(list(batch(x, 2000))[5])

label = x1+x2+x3+x4+x5

results = pd.DataFrame()
results['predicted_acc'] = label
# results['predicted_auc'] = model.predict_proba(x)
results['actual'] = y_valid
results.label.value_counts()

In [63]:
label = x1+x2+x3+x4+x5
results = pd.DataFrame()
results['predicted_acc'] = label
# results['predicted_auc'] = model.predict_proba(x)
results['actual'] = y_valid['label']
results.predicted_acc.value_counts()

In [84]:
len(label), len(y_valid)

In [72]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score([i for i in y_valid['label']], results[['predicted_acc']])
# auc = roc_auc_score(results['actual'], results['predicted_acc'], multi_class='ovr')
# accuracy, auc
accuracy

In [75]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

data = confusion_matrix([i for i in y_valid['label']], results[['predicted_acc']])
# df_cm = pd.DataFrame(data, columns=np.unique([i for i in y_valid['label']]), 
#                      index = np.unique([i for i in y_valid['label']]))
# df_cm.index.name = 'Actual'
# df_cm.columns.name = 'Predicted'
# plt.figure(figsize = (10,7))
# sn.set(font_scale=1.4)#for label size
# sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 16})# font size
data

In [83]:
import seaborn as sns
# sns.heatmap(data, annot=True)

fig, ax = plt.subplots(figsize=(10,10)) 
sns.heatmap(data, linewidths=1, annot=True, ax=ax, fmt='g')

In [ ]:
# to zindiiiiiiiiiiii

# x = [i for i in test['image_path']]
# label = model.predict(x)
# test['label'] = label
# ss['Image_ID'] = test['Image_ID']
# ss['label'] = test['label']
# ss.label.value_counts()

In [ ]:
# 5. Save the model!
# trainer.save_checkpoint("image_classification_model.pt")

# load finetuned checkpoint
# model = ImageClassifier.load_from_checkpoint("image_classification_model.pt")

# predictions = model.predict("path/to/your/own/image.png")


In [ ]:
ss['label'] = list(le.inverse_transform(ss['label']))
ss.to_csv("./submission.csv", index=False)
user.submit(filepaths=["./submission.csv"], comments=["Other"])

In [ ]:
user.submission_board()

In [ ]:
### the gpu crushes so i split the data in batches manually
# import itertools

# def batch(iterable, n=1):
#     l = len(iterable)
#     for ndx in range(0, l, n):
#         yield iterable[ndx:min(ndx + n, l)]

# x = [i for i in test['image_path'] ]
# x1 = model.predict(list(batch(x, 2000))[0])
# x2 = model.predict(list(batch(x, 2000))[1])
# x3 = model.predict(list(batch(x, 2000))[2])
# x4 = model.predict(list(batch(x, 2000))[3])
# x5 = model.predict(list(batch(x, 2000))[4])
# x6 = model.predict(list(batch(x, 2000))[5])

# label = x1+x2+x3+x4+x5+x6

# test['label'] = label
# ss['Image_ID'] = test['Image_ID']
# ss['label'] = test['label']
# ss.label.value_counts()
